In [163]:
%matplotlib inline
from utils import *

In [164]:
train_c = pd.read_parquet('./data/C/train_targets.parquet')

X_train_estimated_c = pd.read_parquet('./data/C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('./data/C/X_train_observed.parquet')
X_test_estimated_c = pd.read_parquet('./data/C/X_test_estimated.parquet')

X_test_estimated_c['date_forecast'] = pd.to_datetime(X_test_estimated_c['date_forecast'])
X_test_estimated_c = X_test_estimated_c[X_test_estimated_c['date_forecast'].dt.minute == 0]

df = pd.concat([X_train_observed_c, X_train_estimated_c])
df = pd.merge(df, train_c, left_on='date_forecast', right_on='time', how='inner')
df = df.drop(columns=['date_forecast', 'date_calc', 'snow_density:kgm3', 'snow_drift:idx', 'fresh_snow_1h:cm', 'fresh_snow_3h:cm', 'fresh_snow_6h:cm', 'fresh_snow_24h:cm'])

In [165]:
imputer = SimpleImputer(strategy='most_frequent')
X_test_estimated_c[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(X_test_estimated_c[['ceiling_height_agl:m', 'cloud_base_agl:m']])
df[['ceiling_height_agl:m', 'cloud_base_agl:m', 'pv_measurement']] = imputer.fit_transform(df[['ceiling_height_agl:m', 'cloud_base_agl:m', 'pv_measurement']])
X_test_estimated_c = X_test_estimated_c.rename(columns={'date_forecast': 'time'})

In [167]:
threshold = 0.95

segments = find_long_constant_periods(train_c['pv_measurement'], threshold=5)
df = remove_constant_periods(df, segments)

df = lag_features_by_one_hour(df, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'])
df = is_estimated(df)
df = remove_highly_correlated_features(df, threshold)

X_test_estimated_c = lag_features_by_one_hour(X_test_estimated_c, ['diffuse_rad_1h:J', 'direct_rad_1h:J', 'clear_sky_energy_1h:J'])
X_test_estimated_c = is_estimated(X_test_estimated_c)
common_columns = df.columns.intersection(X_test_estimated_c.columns)
X_test_estimated_c = X_test_estimated_c.loc[:, common_columns]

C:\Users\marku\AppData\Local\Temp\ipykernel_26580\299797497.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr(method='spearman')


In [168]:
# # Define the split date
# split_date = '2022-10-27'
# 
# # Convert the 'time' column to a datetime object
# df['time'] = pd.to_datetime(df['time'])
# 
# # Sorting the data by the 'time' column to maintain chronological order
# df.sort_values('time', inplace=True)
# 
# # Splitting the data into training and test sets based on the split date
# train_df = df[df['time'] < split_date]
# test_df = df[df['time'] >= split_date]
# 
# # Identifying the features and the target variable
# X_train = train_df.drop(columns=['pv_measurement', 'time'])
# y_train = train_df['pv_measurement']
# X_test = test_df.drop(columns=['pv_measurement', 'time'])
# y_test = test_df['pv_measurement']

In [169]:
train_end_date = '2022-10-21'
# 2023-01-29
validation_end_date = '2023-01-29'
# 2023-03-16
# Convert 'time' column to datetime, if not already
df['time'] = pd.to_datetime(df['time'])

# Split the data into training, validation, and testing sets based on the new split dates
train_df = df[df['time'] < train_end_date]
validation_df = df[(df['time'] >= train_end_date) & (df['time'] < validation_end_date)]
test_df = df[df['time'] >= train_end_date]

# Randomly sample data within these periods (assuming you want to keep the same data structure)
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
validation_df = validation_df.sample(frac=1, random_state=42).reset_index(drop=True)
test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Identifying the features and the target variable
X_train = train_df.drop(columns=['pv_measurement', 'time'])
y_train = train_df['pv_measurement']
X_val = validation_df.drop(columns=['pv_measurement', 'time'])
y_val = validation_df['pv_measurement']
X_test = test_df.drop(columns=['pv_measurement', 'time'])
y_test = test_df['pv_measurement']

In [171]:
# Combine training and validation data into a single dataset for AutoGluon
train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_val, y_val], axis=1)

# Specify the name of the target variable
label = 'pv_measurement'

# Create a TabularPredictor object
predictor = TabularPredictor(label=label, eval_metric="mean_absolute_error").fit(train_data=train_data, tuning_data=val_data, presets='medium_quality', use_bag_holdout=True)

No path specified. Models will be saved in: "AutogluonModels\ag-20231023_210132\"
Presets specified: ['good_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231023_210132\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   953.60 GB / 2047.46 GB (46.6%)
Train Data Rows:    29194
Train Data Columns: 33
Tuning Data Rows:    1304
Tuning Data Columns: 33
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (999.6, 0.0, 65.22502, 156.16072)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may spe

In [172]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L2  -3.855922       1.396158  172.832480                0.000000           0.172312            2       True         10
1          ExtraTreesMSE_BAG_L1  -3.939135       0.616113    1.087869                0.616113           1.087869            1       True          5
2        RandomForestMSE_BAG_L1  -3.972927       0.536000    6.318659                0.536000           6.318659            1       True          3
3         NeuralNetTorch_BAG_L1  -4.017915       0.244045  165.253640                0.244045         165.253640            1       True          8
4           WeightedEnsemble_L3  -4.463264      93.827456  886.017106                0.000000           0.167528            3       True         20
5             LightGBMXT_BAG_L2  -4.499108      92

C:\Users\marku\Desktop\Envs\MLenv\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [173]:
y_pred = predictor.predict(X_test_estimated_c)
y_pred = y_pred.clip(lower=0)
y_pred = y_pred.reset_index(drop=True)
y_pred.index.name = 'id'

In [174]:
df = pd.DataFrame(y_pred)
df.to_csv('result_c.csv')